In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
using Revise
using CreditRisk
using PyPlot
using Distributions
import Random

### Example showing necessity of mixture of factor shifts

In [ ]:
m = 1000
t = 2
pk = 0.05
lk = 1
q = 0.3

a₁ = 0.7
a₂ = 0.65
b₁ = √0.51
b₂ = √0.57

α₁ = 1 - m^(-1/3)
α₂ = 1 - (log(m))^(-1/2)

Gh₁ = (α₁*invnormcdf(1-pk) + α₂*b₁*invnormcdf(q))/a₁
Gh₂ = (α₁*invnormcdf(1-pk) + α₂*b₂*invnormcdf(q))/a₂

Gh₁,Gh₂

### Construct `Parameter`

In [ ]:
N = 1000  # m
C = 2     # 2
S = 2     # d
l = 0.3   # q

cmm = zeros(N,C)
p = 0.05
cmm[:,1] .= p
cmm[:,2] .= 1-p

ead = zeros(N) .+ 1
lgc = zeros(N,C)
lgc[:,1] .= 1
lgc[:,2] .= 0
cn = fill(2,N)

β = zeros(N,C)
β[1:2:N,:] .= [0.7 0]
β[2:2:N,:] .= [0 0.65]
denom = @. sqrt(1 - $sum((β).^2, dims=2))
denom = dropdims(denom; dims=2)

# H[n, c] = inverse_unit_Gaussian(∑ᵧ cmm[c(n), γ])
cum_cmm = cumsum(cmm, dims=2)
H = invnormcdf.(cum_cmm)

weights = zeros(N,C)
weights = ead ./ sum(ead)
weights = weights .* lgc

param = Parameter(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights)

#### Visualize inner level objective function

In [ ]:
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

Ψ = init_Ψ()

Zdist = MvNormal(S, 1)
Z = zeros(S)
pnc = zeros(N, C)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
twist = zeros(N, C)
mgf = zeros(N)
qnc = zeros(N, C)

Random.rand!(Zdist, Z)
@. phi = normcdf((H - $(β*Z)) / denom)
diff!(pnc, phi0; dims=2)

objective(θ) = begin
    θ = θ[1]
    Ψ(θ, pnc, weights) - θ*l
end

xs = -10000:10000
ys = [objective([x]) for x in xs]
display(plot(xs, ys))

### plot the upper bound $F_x(z)$ of $log P(L>x | Z=z)$


Tail bound approximation 

$$
P(L>l | Z=z) \leq e^{F_x(z)}
$$
The upper bond given by 
$$
Fₓ(z) = -θ_m(z)l + \psi(θ_m(z),z)
\quad\quad
\psi(\theta,z) = \sum_{n=1}^{N} \ln \sum_{c=1}^C p_n^c (z) e^{\theta w_n^c}
$$
where
$$
θ_m(z) = \underset{\theta\geq 0}{argmin} \{ -\theta l + \psi(\theta,z) \}
$$

In GL2005, outer level twisting amounts to finding `z` that

$$
\underset{z}{max} P(L>x | Z=z) e^{-\frac{1}{2}z^Tz}
\quad
\rightarrow
\quad
\underset{z}{max} \{F_x(z) - \frac{1}{2} z^Tz\}
$$

Here we want to plot 
$$
\begin{align*}
obj_1 &= F_x(z) \\
obj_2 &= e^{F_x(z) - \frac{1}{2} z^Tz} \\ 
obj_3 &= -log(-F_x(z) + \frac{1}{2} z^Tz)
\end{align*}
$$
Note $obj_1$ is the minimal value for the inner objective function, $obj_2$ is the proxy used for the zero-IS distribution

In [ ]:
(N, C, S, loss, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
pnc = zeros(N, C)
Ψ = init_Ψ()
innerlevel = InnerLevelTwisting(N, C)

# xs = -span:step:span
# ys = -span:step:span

span = -3     # [0, range]
step = -0.025

xs = 0:step:span
ys = 0:step:span

θs   = zeros(length(xs),length(ys))
obj₁ = zeros(length(xs),length(ys))
obj₂ = zeros(length(xs),length(ys))
obj₃ = zeros(length(xs),length(ys))

Fₓ(z) = begin
    @. phi = normcdf((H - $(β*z)) / denom)
    diff!(pnc, phi0; dims=2)
    twist!(innerlevel, pnc, weights, loss)
    θ = get_result(innerlevel)
    Fxz = -θ*l + Ψ(θ, pnc, weights)
    return θ, Fxz # minimizer, f(minimizer)
    end

for i in 1:length(xs)
    for j in 1:length(ys)
        z = [xs[i]; ys[j]]
        θ, Fxz = Fₓ(z)
        θs[i,j] = θ
        obj₁[i,j] = Fxz
        obj₂[i,j] = ℯ^(obj₁[i,j] - 0.5z'z)
        obj₃[i,j] = -log10(-obj₁[i,j] + 0.5z'z)
    end
end

In [ ]:
outerlevelsurf(fig,xs,ys,zs,layout,title) = begin
    nr,nc,nfig = map(x -> parse(Int64,x), collect(layout))
    ax = fig.add_subplot(nr,nc,nfig,projection="3d")
    ax[:view_init](elev=30, azim=45+10)
    sf = plot_surface(xs, ys, zs, rstride=2,edgecolors="k", cstride=2, cmap=ColorMap("summer"), alpha=0.8, linewidth=0.25)
    ct = contour(xs, ys, zs, extend3d=false,zdir="z",offset=minimum(zs),stride=0.2)
    clabel(ct, inline=1, fontsize=10)
    xlabel("z₁"); ylabel("z₂")
    PyPlot.title(title)
    tight_layout()
    fig[:colorbar](sf,shrink=0.2)
end

fig = figure("plots",figsize=(20,16))
outerlevelsurf(fig,xs,ys,θs,  "221","θ_m")
outerlevelsurf(fig,xs,ys,obj₁,"222","Fₓ(z) = -θ_m*l + ψ(θ_m, z)")
outerlevelsurf(fig,xs,ys,obj₂,"223","exp(Fₓ(z) - 0.5*z'z)")
outerlevelsurf(fig,xs,ys,obj₃,"224","-log(-Fₓ(z) + 0.5*z'z)")

In [ ]:
stringbuf2array(str) = begin
    l = split(str, "\n")
    filter!(x -> x ≠ "", l)
    return map(x -> parse(Float64,x), l)
end

nz, ne = 1000,1000
estimates = Dict()

# crude monte carlo
io = IOBuffer()
@time CreditRisk.bernoulli_mc(param, (nz, ne), io)
estimates["CMC"] = stringbuf2array(String(take!(io)))

# GL with 1 shift
io = IOBuffer()
@time CreditRisk.gl2005_mc(param, (nz, ne), ([-2.49748; -0.46698],nothing, 1), io)
estimates["MC1"] = stringbuf2array(String(take!(io)))

# mixture of shifts
io = IOBuffer()
μs = [1.7834 0; 0 1.8977]
@time CreditRisk.gl2008_mc(param, (500,500), (μs, nothing, 1), io)
estimates["MIS"] = stringbuf2array(String(take!(io)))

""

In [ ]:
for (algo,vs) in estimates
    is = 1:length(vs)
    xs = is.*500
    plot(xs[1:200],vs[1:200],label=algo)
end
xlabel("number of iterations")
ylabel("P(L>l)")
ylim(0.005,0.02)
grid("on")
title("different algos' performance")
legend()